In [1]:
import math
import numpy as np
from numba import njit
import random
import sys


## Q1: The stock market

(This is about numba)

A Markov Chain is defined as a sequence of random variables where a parameter depends *only* on the preceding value. This is a crucial tool in statistics, widely used in science and beyond (economics for instance).

For instance, the stock market has phases of growing prices (bull), dreasing prices (bear) and recession. This would be a Marov Chain model:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Finance_Markov_chain_example_state_space.svg/400px-Finance_Markov_chain_example_state_space.svg.png)

where the numbers on the arrows indicate the probabily that the next day will be in a given state.

Your task is to simulate the stock market according to this rule. Start from a random state and simulate many many  iterations. If your code is right, the fraction of days in each state should converge. 

Implement a pure-python version and a numba version, and compare speeds. 


In [2]:
def fractions(N):
    state = np.random.randint(0,3) #select initial random state between 0,1,2
    #print(state)
    days_0 = 0. #days in state 0
    days_1 = 0.
    days_2 = 0. #total must be N
    for i in range(N): #cycle over the iterations
        
        number = np.random.uniform(0,1) #gives the probability
        if state == 0: #bull
            if number < 0.9:
                state = 0
                days_0 = days_0 + 1
            elif number >= 0.9 and number < 0.975:
                state = 1
                days_1 = days_1 + 1
            elif number >= 0.975:
                state = 2
                days_2 = days_2 + 1
        elif state == 1: #bear
            if number < 0.8:
                state = 1
                days_1 = days_1 + 1
            elif number >= 0.8 and number < 0.95:
                state = 0
                days_0 = days_0 + 1
            elif number >= 0.95:
                state = 2
                days_2 = days_2 + 1
        elif state == 2: #stagnant
            if number < 0.5:
                state = 2
                days_2 = days_2 + 1
            elif number >= 0.5 and number < 0.75:
                state = 0
                days_0 = days_0 + 1
            elif number >= 0.75:
                state = 1
                days_1 = days_1 + 1
    frac_0 = days_0/N
    frac_1 = days_1/N
    frac_2 = days_2/N #sum should be 1
    #return days_0, days_1, days_2, frac_0, frac_1, frac_2
    return frac_0, frac_1, frac_2          

In [3]:
c_fractions = njit(fractions)

In [23]:
N = 1000000
frac_0, frac_1, frac_2  = c_fractions(N)

# see fractions in each day
print(f'The fractions of days in each state converged to: {frac_0} in state bull(0), {frac_1} in state bear (1) and {frac_2} in state stagnant (2)')

# check sum of days = N
print('The total number of days is: ', frac_0*N + frac_1*N + frac_2*N)

The fractions of days in each state converged to: 0.623687 in state bull(0), 0.313522 in state bear (1) and 0.062791 in state stagnant (2)
The total number of days is:  1000000.0


In [24]:
# time the two functions 

%timeit fractions(N)
%timeit c_fractions(N)

695 ms ± 3.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.69 ms ± 4.93 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Q2: Consistent plotting

(This is about python's decorators)

Write a decorator for the plots of all your papers. 

- Remember a decorator takes a function. 
- This function in turn should return a matplotlib figure object.
- Before the function is called, the decorator should intialize a matplotlib figure with the options that you like the most (fontsize, ticks, etc etc)
- After the figure is done, the decorator should save it to pdf.

This is a great hack for your papers! You do this once and for all, and all plots in your paper will be beautiful, all with the same style/fontsize/etc. All you'll need to do is adding `@myplot` to the relevant plotting functions. 
    

In [2]:
import matplotlib.pyplot as plt

def myplot(func):
    
    def wrapper(*args, **kwargs):
        
        # create the figure with preferences
        fig, ax = plt.subplots(figsize=(7, 7))
        plt.rcParams.update({
            
            'axes.labelsize':15,
            'axes.titlesize':20,
            'legend.fontsize':10,
            'legend.loc':'best',
            'xtick.labelsize':14,
            'ytick.labelsize':14,
            'xtick.major.size':5,
            'ytick.major.size':5,
            'xtick.minor.size':4,
            'ytick.minor.size':4,
            'xtick.direction':'in',
            'ytick.direction':'in',
            'xtick.bottom':True,
            'xtick.top':True,
            'ytick.left':True,
            'ytick.right':True,
            'lines.linewidth':2,
            'figure.autolayout': True
            
        })
        ax.spines['bottom'].set_linewidth(2.)
        ax.spines['top'].set_linewidth(2.)
        ax.spines['right'].set_linewidth(2.)
        ax.spines['left'].set_linewidth(2.)

        # call the wrapped function for the plot
        plot = func(*args, **kwargs)

        # save the plot to PDF
        fig.savefig('plot.pdf', bbox_inches='tight')
        plt.close(fig) 

        return plot

    return wrapper



In [4]:
# example with a raw function

@myplot
def raw_plot():
    # Your plot generation code here
    x = np.arange(0,100,1)
    y = np.sin(x)
    plt.plot(x, y, label='my function')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('My fancy plot')
    plt.legend()

raw_plot()